In [ ]:
from scipy.io import wavfile
import numpy as np
import os
from scipy.io.wavfile import read
from pystoi import stoi
from logmmse import logmmse_from_file
from tqdm import tqdm

clean_folderpath = "/Users/anude/Desktop/Processed_Data/test/clean/SNR_-5db"  # directory of clean files
noisy_folderpath = "/Users/anude/Desktop/Processed_Data/test/noisy/SNR_-5db"  # directory of noisy files

cwd = os.getcwd()
output_filepath = os.environ["HOME"]
output_filepath = os.path.join(output_filepath, "output.wav")  # set as a random file in your desktop (non-permanent storage of created sound files)

cleanfiles = []
noisyfiles = []

for filename in os.listdir(clean_folderpath):
    file_path = os.path.join(clean_folderpath, filename)
    cleanfiles.append(file_path)

for filename in os.listdir(noisy_folderpath):
    file_path = os.path.join(noisy_folderpath, filename)
    noisyfiles.append(file_path)

folderlength = len(cleanfiles)  # assumption: # of clean files = # of noisy files

stois = []

for i in tqdm(range(folderlength), desc="Processing files"):
    print(f'Processing cleanfile: {cleanfiles[i]} and noisy file: {noisyfiles[i]}')
    noisyfile = noisyfiles[i]
    cleanfile = cleanfiles[i]

    try:
        # feed the noisy sound file in with the output2, so a denoised (from logmmse) sound file is written in the directory of output2
        denoised = logmmse_from_file(noisyfile)  # THIS IS THE NEW LOGMMSE

        # read in properties/data from the noisy and denoised sound files
        fs, noisy = read(noisyfile)
        # denoised is in the same format as noisy

        # Ensure both signals have the same length by truncating to the minimum length
        min_length = min(len(noisy), len(denoised))
        noisy = noisy[:min_length]
        denoised = denoised[:min_length]

        # Calculate STOI and check if it's valid
        d = stoi(noisy, denoised, fs, extended=False)

        if not np.isnan(d):
            print(f"STOI for file {i}: {d}")
            stois.append(d)
        else:
            print(f"STOI calculation failed for file {i}")
    except ValueError as e:
        print(f"Error processing file {i}: {e}")
        continue  # Continue to the next file

# Calculate the average STOI for valid entries in the stois list
valid_stois = [s for s in stois if not np.isnan(s)]
if valid_stois:
    print("The average STOI is: ", np.average(valid_stois))
else:
    print("No valid STOI values available.")
    
#The average STOI is:  0.7710802727988904
